##### Prawa autorskie 2021 Autorzy TensorFlow.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Uwaga: Możesz uruchomić ten przykład już teraz w notatniku w stylu Jupytera, nie jest wymagana żadna konfiguracja! Po prostu kliknij „Uruchom w Google Colab”

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/model_analysis/tfma_basic"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Zobacz na TensorFlow.org</a></td>
<td><a target="_blank" href="https://colab.sandbox.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/model_analysis/tfma_basic.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Uruchom w Google Colab</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/tfx/blob/master/docs/tutorials/model_analysis/tfma_basic.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Zobacz źródło na GitHub</a></td>
<td><a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/tfx/docs/tutorials/model_analysis/tfma_basic.ipynb"><img width="32px" src="https://www.tensorflow.org/images/download_logo_32px.png">Pobierz notatnik</a></td>
</table></div>

# Analiza modelu TensorFlow

***Przykład kluczowego komponentu TensorFlow Extended (TFX)***

[TensorFlow Model Analysis (TFMA)](https://www.tensorflow.org/tfx/guide/tfma) to biblioteka do przeprowadzania oceny modeli w różnych segmentach danych. TFMA wykonuje swoje obliczenia w sposób rozproszony na dużych ilościach danych przy użyciu [Apache Beam](https://beam.apache.org/documentation/programming-guide/) .

Ten przykładowy notatnik colab ilustruje, w jaki sposób można wykorzystać TFMA do badania i wizualizacji wydajności modelu w odniesieniu do charakterystyki zbioru danych. Użyjemy modelu, który trenowaliśmy wcześniej, a teraz możesz pobawić się wynikami! Model, który przeszkoliliśmy, dotyczył [Chicago Taxi Example](https://github.com/tensorflow/tfx/tree/master/tfx/examples/chicago_taxi_pipeline) , który używa [zestawu danych Taxi Trips](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew) opublikowanego przez miasto Chicago. Przeglądaj pełny zestaw danych w [interfejsie użytkownika BigQuery](https://bigquery.cloud.google.com/dataset/bigquery-public-data:chicago_taxi_trips) .

Jako twórca modeli i programista zastanów się, w jaki sposób wykorzystywane są te dane oraz jakie potencjalne korzyści i szkody mogą spowodować prognozy modelu. Taki model mógłby wzmocnić społeczne uprzedzenia i różnice. Czy funkcja jest istotna dla problemu, który chcesz rozwiązać, czy też wprowadzi błąd? Aby uzyskać więcej informacji, przeczytaj o <a target="_blank" href="https://developers.google.com/machine-learning/fairness-overview/">uczciwości ML</a> .

Uwaga: Aby zrozumieć TFMA i sposób, w jaki działa z Apache Beam, musisz wiedzieć trochę o samym Apache Beam. <a target="_blank" href="https://beam.apache.org/documentation/programming-guide/">Przewodnik programowania wiązki</a> to świetne miejsce do rozpoczęcia.

Kolumny w zbiorze danych to:

<table>
<tr>
<td>obszar_społeczności_odbioru</td>
<td>opłata</td>
<td>podróż_początkowa_miesiąc</td>
</tr>
<tr>
<td>godzina_rozpoczęcia_wycieczki</td>
<td>dzień_rozpoczęcia_wycieczki</td>
<td>trip_start_timestamp</td>
</tr>
<tr>
<td>szerokość_podbierania</td>
<td>długość_geograficzna odbioru</td>
<td>zrzut_szerokość geograficzna</td>
</tr>
<tr>
<td>dropoff_longitude</td>
<td>podróż_mile</td>
<td>pickup_census_tract</td>
</tr>
<tr>
<td>dropoff_census_tract</td>
<td>typ płatności</td>
<td>firma</td>
</tr>
<tr>
<td>podróż_sekund</td>
<td>dropoff_community_area</td>
<td>porady</td>
</tr>
</table>

## Zainstaluj rozszerzenia Jupyter

Uwaga: w przypadku uruchamiania w lokalnym notatniku Jupyter te rozszerzenia Jupyter muszą zostać zainstalowane w środowisku przed uruchomieniem Jupyter.

```bash
jupyter nbextension enable --py widgetsnbextension --sys-prefix
jupyter nbextension install --py --symlink tensorflow_model_analysis --sys-prefix
jupyter nbextension enable --py tensorflow_model_analysis --sys-prefix
```

## Zainstaluj analizę modelu TensorFlow (TFMA)

Spowoduje to pobranie wszystkich zależności i zajmie minutę.


In [ ]:
# Upgrade pip to the latest, and install TFMA.
!pip install -U pip
!pip install tensorflow-model-analysis

**Teraz musisz ponownie uruchomić środowisko wykonawcze przed uruchomieniem poniższych komórek.**

In [ ]:
# This setup was tested with TF 2.10 and TFMA 0.41 (using colab), but it should
# also work with the latest release.
import sys

# Confirm that we're using Python 3
assert sys.version_info.major==3, 'This notebook must be run using Python 3.'

import tensorflow as tf
print('TF version: {}'.format(tf.__version__))
import apache_beam as beam
print('Beam version: {}'.format(beam.__version__))
import tensorflow_model_analysis as tfma
print('TFMA version: {}'.format(tfma.__version__))

**UWAGA: Powyższe dane wyjściowe powinny być wolne od błędów przed kontynuowaniem. Uruchom ponownie instalację, jeśli nadal widzisz błędy. Pamiętaj też o ponownym uruchomieniu środowiska uruchomieniowego/jądra przed przejściem do następnego kroku.**

## Załaduj pliki

Pobierzemy plik tar, który zawiera wszystko, czego potrzebujemy. To obejmuje:

- Zbiory danych szkoleniowych i ewaluacyjnych
- Schemat danych
- Trenowanie i obsługa zapisanych modeli (keras i estymator) oraz ewaluacja zapisanych modeli (estymator).

In [ ]:
# Download the tar file from GCP and extract it
import io, os, tempfile
TAR_NAME = 'saved_models-2.2'
BASE_DIR = tempfile.mkdtemp()
DATA_DIR = os.path.join(BASE_DIR, TAR_NAME, 'data')
MODELS_DIR = os.path.join(BASE_DIR, TAR_NAME, 'models')
SCHEMA = os.path.join(BASE_DIR, TAR_NAME, 'schema.pbtxt')
OUTPUT_DIR = os.path.join(BASE_DIR, 'output')

!curl -O https://storage.googleapis.com/artifacts.tfx-oss-public.appspot.com/datasets/{TAR_NAME}.tar
!tar xf {TAR_NAME}.tar
!mv {TAR_NAME} {BASE_DIR}
!rm {TAR_NAME}.tar

print("Here's what we downloaded:")
!ls -R {BASE_DIR}

## Przeanalizuj schemat

Wśród rzeczy, które pobraliśmy, był schemat dla naszych danych, który został utworzony przez [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started/) . Przeanalizujmy to teraz, abyśmy mogli użyć go z TFMA.

In [ ]:
import tensorflow as tf
from google.protobuf import text_format
from tensorflow.python.lib.io import file_io
from tensorflow_metadata.proto.v0 import schema_pb2
from tensorflow.core.example import example_pb2

schema = schema_pb2.Schema()
contents = file_io.read_file_to_string(SCHEMA)
schema = text_format.Parse(contents, schema)

## Użyj schematu, aby utworzyć TFRecords

Musimy dać TFMA dostęp do naszego zbioru danych, więc utwórzmy plik TFRecords. Możemy użyć naszego schematu, aby go utworzyć, ponieważ daje nam poprawny typ dla każdej funkcji.

In [ ]:
import csv

datafile = os.path.join(DATA_DIR, 'eval', 'data.csv')
reader = csv.DictReader(open(datafile, 'r'))
examples = []
for line in reader:
  example = example_pb2.Example()
  for feature in schema.feature:
    key = feature.name
    if feature.type == schema_pb2.FLOAT:
      example.features.feature[key].float_list.value[:] = (
          [float(line[key])] if len(line[key]) > 0 else [])
    elif feature.type == schema_pb2.INT:
      example.features.feature[key].int64_list.value[:] = (
          [int(line[key])] if len(line[key]) > 0 else [])
    elif feature.type == schema_pb2.BYTES:
      example.features.feature[key].bytes_list.value[:] = (
          [line[key].encode('utf8')] if len(line[key]) > 0 else [])
  # Add a new column 'big_tipper' that indicates if tips was > 20% of the fare. 
  # TODO(b/157064428): Remove after label transformation is supported for Keras.
  big_tipper = float(line['tips']) > float(line['fare']) * 0.2
  example.features.feature['big_tipper'].float_list.value[:] = [big_tipper]
  examples.append(example)

tfrecord_file = os.path.join(BASE_DIR, 'train_data.rio')
with tf.io.TFRecordWriter(tfrecord_file) as writer:
  for example in examples:
    writer.write(example.SerializeToString())

!ls {tfrecord_file}

## Skonfiguruj i uruchom TFMA

TFMA obsługuje wiele różnych typów modeli, w tym modele TF keras, modele oparte na ogólnych interfejsach API sygnatur TF2, a także modele oparte na estymatorach TF. Przewodnik [get_started](https://www.tensorflow.org/tfx/model_analysis/get_started) zawiera pełną listę obsługiwanych typów modeli i wszelkie ograniczenia. W tym przykładzie pokażemy, jak skonfigurować model oparty na keras, a także model oparty na estymatorze, który został zapisany jako [`EvalSavedModel`](https://www.tensorflow.org/tfx/model_analysis/eval_saved_model) . Zobacz [często zadawane pytania](https://www.tensorflow.org/tfx/model_analysis/faq) , aby zobaczyć przykłady innych konfiguracji.

TFMA zapewnia obsługę obliczania metryk, które były używane w czasie szkolenia (tj. metryk wbudowanych), a także metryk zdefiniowanych po zapisaniu modelu jako część ustawień konfiguracyjnych TFMA. W przypadku naszej [konfiguracji](https://www.tensorflow.org/tfx/model_analysis/setup) keras zademonstrujemy ręczne dodawanie naszych metryk i wykresów w ramach naszej konfiguracji (zobacz przewodnik [po metrykach](https://www.tensorflow.org/tfx/model_analysis/metrics) , aby uzyskać informacje na temat obsługiwanych metryk i wykresów). Do konfiguracji estymatora użyjemy wbudowanych metryk zapisanych wraz z modelem. Nasze konfiguracje obejmują również szereg specyfikacji krojenia, które omówiono bardziej szczegółowo w poniższych sekcjach.

Po utworzeniu [`tfma.EvalConfig`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalConfig) i [`tfma.EvalSharedModel`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalSharedModel) możemy następnie uruchomić TFMA za pomocą [`tfma.run_model_analysis`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/run_model_analysis) . Spowoduje to utworzenie [`tfma.EvalResult`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalResult) , którego możemy później użyć do renderowania naszych metryk i wykresów.

### Keras

In [ ]:
import tensorflow_model_analysis as tfma

# Setup tfma.EvalConfig settings
keras_eval_config = text_format.Parse("""
  ## Model information
  model_specs {
    # For keras (and serving models) we need to add a `label_key`.
    label_key: "big_tipper"
  }

  ## Post training metric information. These will be merged with any built-in
  ## metrics from training.
  metrics_specs {
    metrics { class_name: "ExampleCount" }
    metrics { class_name: "AUC" }
    metrics { class_name: "Precision" }
    metrics { class_name: "Recall" }
    metrics { class_name: "MeanPrediction" }
    metrics { class_name: "Calibration" }
    metrics { class_name: "CalibrationPlot" }
    metrics { class_name: "ConfusionMatrixPlot" }
    # ... add additional metrics and plots ...
  }

  ## Slicing information
  slicing_specs {}  # overall slice
  slicing_specs {
    feature_keys: ["trip_start_hour"]
  }
  slicing_specs {
    feature_keys: ["trip_start_day"]
  }
  slicing_specs {
    feature_values: {
      key: "trip_start_month"
      value: "1"
    }
  }
""", tfma.EvalConfig())

# Create a tfma.EvalSharedModel that points at our keras model.
keras_model_path = os.path.join(MODELS_DIR, 'keras', '2')
keras_eval_shared_model = tfma.default_eval_shared_model(
    eval_saved_model_path=keras_model_path,
    eval_config=keras_eval_config)

keras_output_path = os.path.join(OUTPUT_DIR, 'keras')

# Run TFMA
keras_eval_result = tfma.run_model_analysis(
    eval_shared_model=keras_eval_shared_model,
    eval_config=keras_eval_config,
    data_location=tfrecord_file,
    output_path=keras_output_path)

### Taksator

In [ ]:
import tensorflow_model_analysis as tfma

# Setup tfma.EvalConfig settings
estimator_eval_config = text_format.Parse("""
  ## Model information
  model_specs {
    # To use EvalSavedModel set `signature_name` to "eval".
    signature_name: "eval"
  }

  ## Post training metric information. These will be merged with any built-in
  ## metrics from training.
  metrics_specs {
    metrics { class_name: "ConfusionMatrixPlot" }
    # ... add additional metrics and plots ...
  }

  ## Slicing information
  slicing_specs {}  # overall slice
  slicing_specs {
    feature_keys: ["trip_start_hour"]
  }
  slicing_specs {
    feature_keys: ["trip_start_day"]
  }
  slicing_specs {
    feature_values: {
      key: "trip_start_month"
      value: "1"
    }
  }
""", tfma.EvalConfig())

# Create a tfma.EvalSharedModel that points at our eval saved model.
estimator_base_model_path = os.path.join(
    MODELS_DIR, 'estimator', 'eval_model_dir')
estimator_model_path = os.path.join(
    estimator_base_model_path, os.listdir(estimator_base_model_path)[0])
estimator_eval_shared_model = tfma.default_eval_shared_model(
    eval_saved_model_path=estimator_model_path,
    eval_config=estimator_eval_config)

estimator_output_path = os.path.join(OUTPUT_DIR, 'estimator')

# Run TFMA
estimator_eval_result = tfma.run_model_analysis(
    eval_shared_model=estimator_eval_shared_model,
    eval_config=estimator_eval_config,
    data_location=tfrecord_file,
    output_path=estimator_output_path)

## Wizualizacja metryk i wykresów

Teraz, gdy przeprowadziliśmy ocenę, spójrzmy na nasze wizualizacje przy użyciu TFMA. W poniższych przykładach zwizualizujemy wyniki przeprowadzenia oceny na modelu keras. Aby wyświetlić model oparty na estymatorze, zaktualizuj `eval_result_path` tak, aby wskazywała na naszą zmienną `estimator_output_path` .

In [ ]:
eval_result_path = keras_output_path
# eval_result_path = estimator_output_path

eval_result = keras_eval_result
# eval_result = estimator_eval_result

### Metryki renderowania

TFMA udostępnia interfejsy API ramek danych w [`tfma.experimental.dataframe`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/experimental) do ładowania zmaterializowanych danych wyjściowych jako [`Pandas DataFrames`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) . Aby wyświetlić metryki, możesz użyć `metrics_as_dataframes(tfma.load_metrics(eval_path))` , która zwraca obiekt, który potencjalnie zawiera kilka ramek danych, po jednej dla każdego typu wartości metryki ( `double_value` , `confusion_matrix_at_thresholds` , `bytes_value` i `array_value` ). Konkretne wypełnione ramki danych zależą od wyniku eval. Tutaj jako przykład pokazujemy ramkę DataFrame `double_value` .

In [ ]:
import tensorflow_model_analysis.experimental.dataframe as tfma_dataframe
dfs = tfma_dataframe.metrics_as_dataframes(
  tfma.load_metrics(eval_result_path))

display(dfs.double_value.head())

Każda z ramek danych ma multiindeks kolumny z kolumnami najwyższego poziomu: `slices` , `metric_keys` i `metric_values` . Dokładne kolumny każdej grupy mogą się zmieniać w zależności od ładunku. możemy użyć API `DataFrame.columns` do sprawdzenia wszystkich kolumn z wieloma indeksami. Na przykład kolumny plasterków to „Overall”, „trip_start_day”, „trip_start_hour” i „trip_start_month”, które są konfigurowane przez `slicing_specs` w pliku `eval_config` .

In [ ]:
print(dfs.double_value.columns)

### Automatyczne obracanie

DataFrame jest projektowany szczegółowo, więc nie ma utraty informacji z ładunku. Czasami jednak, do bezpośredniego wykorzystania, możemy chcieć uporządkować informacje w bardziej zwięzłej, ale stratnej formie: wycinki jako wiersze i metryki jako kolumny. W tym celu TFMA udostępnia interfejs API `auto_pivot` . Narzędzie obraca się na wszystkich nieunikatowych kolumnach wewnątrz `metric_keys` i domyślnie skupia wszystkie wycinki w jednej kolumnie `stringified_slices` .

In [ ]:
tfma_dataframe.auto_pivot(dfs.double_value).head()

### Filtrowanie plasterków

Ponieważ dane wyjściowe to DataFrame, dowolne natywne interfejsy API DataFrame mogą być używane do krojenia i krojenia DataFrame. Na przykład, jeśli interesuje nas tylko `trip_start_hour` 1, 3, 5, 7, a nie `trip_start_day` , możemy użyć logiki filtrowania `.loc` DataFrame. Ponownie używamy funkcji `auto_pivot` , aby ponownie zorganizować ramkę DataFrame w widoku plasterek vs. metryki po przeprowadzeniu filtrowania.

In [ ]:
df_double = dfs.double_value
df_filtered = (df_double
  .loc[df_double.slices.trip_start_hour.isin([1,3,5,7])]
)
display(tfma_dataframe.auto_pivot(df_filtered))

### Sortowanie według wartości metrycznych

Możemy również sortować wycinki według wartości metryki. Jako przykład pokazujemy, jak sortować wycinki w powyższej ramce DataFrame, zwiększając AUC, abyśmy mogli znaleźć wycinki o niskiej wydajności. Obejmuje to dwa kroki: automatyczne obracanie, tak aby wycinki były reprezentowane jako wiersze, a kolumny to metryki, a następnie sortowanie przestawnej ramki danych według kolumny AUC.

In [ ]:
# Pivoted table sorted by AUC in ascending order.
df_sorted = (
    tfma_dataframe.auto_pivot(df_double)
    .sort_values(by='auc', ascending=True)
    )
display(df_sorted.head())

### Wykresy renderowania

Wszelkie wykresy, które zostały dodane do `tfma.EvalConfig` jako `metric_specs` po szkoleniu, można wyświetlić za pomocą [`tfma.view.render_plot`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/view/render_plot) .

Podobnie jak w przypadku metryk, wykresy można przeglądać według wycinków. W przeciwieństwie do metryk, można wyświetlić tylko wykresy dla określonej wartości wycinka, więc należy użyć parametru `tfma.SlicingSpec` , który musi określać zarówno nazwę, jak i wartość elementu wycinka. Jeśli nie podano wycinka, używane są wykresy dla wycinka `Overall` .

W poniższym przykładzie wyświetlamy wykresy `CalibrationPlot` i `ConfusionMatrixPlot` , które zostały obliczone dla wycinka `trip_start_hour:1` .

In [ ]:
tfma.view.render_plot(
    eval_result,
    tfma.SlicingSpec(feature_values={'trip_start_hour': '1'}))

## Śledzenie wydajności modelu w czasie

Twój zestaw danych szkoleniowych zostanie użyty do szkolenia Twojego modelu i, miejmy nadzieję, będzie reprezentatywny dla Twojego testowego zestawu danych oraz danych, które zostaną wysłane do Twojego modelu w produkcji. Jednak chociaż dane w żądaniach wnioskowania mogą pozostać takie same jak dane szkoleniowe, w wielu przypadkach zaczną się zmieniać na tyle, że zmieni się wydajność modelu.

Oznacza to, że musisz na bieżąco monitorować i mierzyć wydajność swojego modelu, aby być świadomym zmian i reagować na nie. Przyjrzyjmy się, jak TFMA może pomóc.

Załadujmy 3 różne przebiegi modelu i użyjmy TFMA, aby zobaczyć, jak się porównują, używając [`render_time_series`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/view/render_time_series) .

In [ ]:
# Note this re-uses the EvalConfig from the keras setup.

# Run eval on each saved model
output_paths = []
for i in range(3):
  # Create a tfma.EvalSharedModel that points at our saved model.
  eval_shared_model = tfma.default_eval_shared_model(
      eval_saved_model_path=os.path.join(MODELS_DIR, 'keras', str(i)),
      eval_config=keras_eval_config)

  output_path = os.path.join(OUTPUT_DIR, 'time_series', str(i))
  output_paths.append(output_path)

  # Run TFMA
  tfma.run_model_analysis(eval_shared_model=eval_shared_model,
                          eval_config=keras_eval_config,
                          data_location=tfrecord_file,
                          output_path=output_path)

Najpierw wyobraźmy sobie, że wczoraj wytrenowaliśmy i wdrożyliśmy nasz model, a teraz chcemy zobaczyć, jak sobie radzi z nowymi danymi, które napływają dzisiaj. Wizualizacja rozpocznie się od wyświetlenia AUC. Z poziomu interfejsu użytkownika możesz:

- Dodaj inne dane, korzystając z menu „Dodaj serię danych”.
- Zamknij niechciane wykresy, klikając x
- Najedź kursorem na punkty danych (końce segmentów linii na wykresie), aby uzyskać więcej szczegółów

Uwaga: Na wykresach serii metrycznych oś X to nazwa katalogu modelu badanego przebiegu modelu. Same te nazwy nie mają znaczenia.

In [ ]:
eval_results_from_disk = tfma.load_eval_results(output_paths[:2])

tfma.view.render_time_series(eval_results_from_disk)

Teraz wyobraźmy sobie, że minął kolejny dzień i chcemy zobaczyć, jak radzi sobie z napływającymi dziś nowymi danymi w porównaniu z poprzednimi dwoma dniami:

In [ ]:
eval_results_from_disk = tfma.load_eval_results(output_paths)

tfma.view.render_time_series(eval_results_from_disk)

## Walidacja modelu

TFMA można skonfigurować do jednoczesnej oceny wielu modeli. Zwykle robi się to w celu porównania nowego modelu z linią bazową (taką jak model aktualnie obsługujący) w celu określenia, jakie są różnice wydajności w metrykach (np. AUC itp.) w stosunku do linii bazowej. Po skonfigurowaniu [progów](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/MetricThreshold) TFMA wygeneruje rekord [`tfma.ValidationResult`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/ValidationResult) wskazujący, czy wydajność jest zgodna z oczekiwaniami.

Skonfigurujmy ponownie naszą ocenę keras, aby porównać dwa modele: kandydata i linię bazową. Zweryfikujemy również wyniki kandydata w odniesieniu do poziomu bazowego, ustawiając wartość [`tmfa.MetricThreshold`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/MetricThreshold) dla metryki AUC.

In [ ]:
# Setup tfma.EvalConfig setting
eval_config_with_thresholds = text_format.Parse("""
  ## Model information
  model_specs {
    name: "candidate"
    # For keras we need to add a `label_key`.
    label_key: "big_tipper"
  }
  model_specs {
    name: "baseline"
    # For keras we need to add a `label_key`.
    label_key: "big_tipper"
    is_baseline: true
  }

  ## Post training metric information
  metrics_specs {
    metrics { class_name: "ExampleCount" }
    metrics { class_name: "BinaryAccuracy" }
    metrics { class_name: "BinaryCrossentropy" }
    metrics {
      class_name: "AUC"
      threshold {
        # Ensure that AUC is always > 0.9
        value_threshold {
          lower_bound { value: 0.9 }
        }
        # Ensure that AUC does not drop by more than a small epsilon
        # e.g. (candidate - baseline) > -1e-10 or candidate > baseline - 1e-10
        change_threshold {
          direction: HIGHER_IS_BETTER
          absolute { value: -1e-10 }
        }
      }
    }
    metrics { class_name: "AUCPrecisionRecall" }
    metrics { class_name: "Precision" }
    metrics { class_name: "Recall" }
    metrics { class_name: "MeanLabel" }
    metrics { class_name: "MeanPrediction" }
    metrics { class_name: "Calibration" }
    metrics { class_name: "CalibrationPlot" }
    metrics { class_name: "ConfusionMatrixPlot" }
    # ... add additional metrics and plots ...
  }

  ## Slicing information
  slicing_specs {}  # overall slice
  slicing_specs {
    feature_keys: ["trip_start_hour"]
  }
  slicing_specs {
    feature_keys: ["trip_start_day"]
  }
  slicing_specs {
    feature_keys: ["trip_start_month"]
  }
  slicing_specs {
    feature_keys: ["trip_start_hour", "trip_start_day"]
  }
""", tfma.EvalConfig())

# Create tfma.EvalSharedModels that point at our keras models.
candidate_model_path = os.path.join(MODELS_DIR, 'keras', '2')
baseline_model_path = os.path.join(MODELS_DIR, 'keras', '1')
eval_shared_models = [
  tfma.default_eval_shared_model(
      model_name=tfma.CANDIDATE_KEY,
      eval_saved_model_path=candidate_model_path,
      eval_config=eval_config_with_thresholds),
  tfma.default_eval_shared_model(
      model_name=tfma.BASELINE_KEY,
      eval_saved_model_path=baseline_model_path,
      eval_config=eval_config_with_thresholds),
]

validation_output_path = os.path.join(OUTPUT_DIR, 'validation')

# Run TFMA
eval_result_with_validation = tfma.run_model_analysis(
    eval_shared_models,
    eval_config=eval_config_with_thresholds,
    data_location=tfrecord_file,
    output_path=validation_output_path)

Podczas przeprowadzania ewaluacji z jednym lub kilkoma modelami względem linii bazowej, TFMA automatycznie dodaje metryki diff dla wszystkich metryk obliczonych podczas ewaluacji. Nazwy tych metryk pochodzą od odpowiednich metryk, ale z `_diff` dołączonym do nazwy metryki.

Przyjrzyjmy się metrykom wygenerowanym przez nasz przebieg:

In [ ]:
tfma.view.render_time_series(eval_result_with_validation)

Teraz spójrzmy na dane wyjściowe z naszych kontroli poprawności. Aby wyświetlić wyniki walidacji, używamy [`tfma.load_validator_result`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/load_validation_result) . W naszym przykładzie sprawdzanie poprawności kończy się niepowodzeniem, ponieważ AUC jest poniżej progu.

In [ ]:
validation_result = tfma.load_validation_result(validation_output_path)
print(validation_result.validation_ok)

# Copyright © 2020 Autorzy TensorFlow.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Uwaga: Ta witryna zawiera aplikacje korzystające z danych, które zostały zmodyfikowane do użytku z oryginalnego źródła, www.cityofchicago.org, oficjalnej witryny internetowej miasta Chicago. Miasto Chicago nie zgłasza żadnych roszczeń co do treści, dokładności, aktualności ani kompletności jakichkolwiek danych udostępnianych na tej stronie. Dane podane na tej stronie mogą ulec zmianie w dowolnym momencie. Rozumie się, że dane podane na tej stronie są wykorzystywane na własne ryzyko.